In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
print(os.listdir("../input"))

['ey-nextwave', 'ndscindexes']


In [2]:
df_train = pd.read_csv("../input/ey-nextwave/data_train/data_train.csv")
df_test = pd.read_csv("../input/ey-nextwave/data_test/data_test.csv")

In [3]:
df_train.tail()

,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
814257,814257,ffffc6359725f0e1feac9ef1872ab207_11,traj_ffffc6359725f0e1feac9ef1872ab207_11_4,02:21:11,02:21:11,NaN,NaN,NaN,3.744666e+06,-1.925679e+07,3.744666e+06,-1.925679e+07
814258,814258,ffffc6359725f0e1feac9ef1872ab207_11,traj_ffffc6359725f0e1feac9ef1872ab207_11_5,06:02:17,06:02:17,NaN,NaN,NaN,3.744732e+06,-1.925614e+07,3.744732e+06,-1.925614e+07
814259,814259,ffffc6359725f0e1feac9ef1872ab207_11,traj_ffffc6359725f0e1feac9ef1872ab207_11_7,09:52:13,09:52:13,NaN,NaN,NaN,3.744666e+06,-1.925679e+07,3.744666e+06,-1.925679e+07
814260,814260,ffffc6359725f0e1feac9ef1872ab207_11,traj_ffffc6359725f0e1feac9ef1872ab207_11_8,14:20:26,14:27:15,NaN,NaN,NaN,3.741043e+06,-1.929051e+07,3.741057e+06,-1.928936e+07
814261,814261,ffffc6359725f0e1feac9ef1872ab207_11,traj_ffffc6359725f0e1feac9ef1872ab207_11_10,14:56:46,15:04:05,0.0,0.0,0.0,3.743948e+06,-1.923560e+07,3.744842e+06,-1.926218e+07


In [4]:
df_test.tail()

,Unnamed: 0,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
202932,202932,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_3,11:23:33,11:23:33,NaN,NaN,NaN,3.762713e+06,-1.935493e+07,3.762713e+06,-1.935493e+07
202933,202933,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_4,12:12:10,12:12:10,NaN,NaN,NaN,3.761040e+06,-1.935274e+07,3.761040e+06,-1.935274e+07
202934,202934,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_5,13:08:14,13:12:01,NaN,NaN,NaN,3.762680e+06,-1.935570e+07,3.762683e+06,-1.935529e+07
202935,202935,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_6,14:14:36,14:14:36,NaN,NaN,NaN,3.761776e+06,-1.935772e+07,3.761776e+06,-1.935772e+07
202936,202936,fff9552047b095e8242b4913f3289a26_25,traj_fff9552047b095e8242b4913f3289a26_25_7,15:30:54,15:30:54,NaN,NaN,NaN,3.762713e+06,-1.935478e+07,NaN,NaN


In [5]:
# normalising location information
X_MIN = 3750901.5068
X_MAX = 3770901.5068
X_MID = X_MIN + 0.5 * (X_MAX - X_MIN)
Y_MIN = -19268905.6133
Y_MAX = -19208905.6133
Y_MID = Y_MIN + 0.5 * (Y_MAX - Y_MIN)

def normalise_X(arr):
    return (arr - X_MID) / 10000

def normalise_Y(arr):
    return (arr - Y_MID) / 100000  
    # extra zero by design, seems to make figure to be in proportion
    # looking for evidence that the x-axis and y-axis fulfil some ratio

x_min, x_max = normalise_X(X_MIN), normalise_X(X_MAX)
y_min, y_max = normalise_Y(Y_MIN), normalise_Y(Y_MAX)
print("Borders:")
print("{:.4f} < X < {:.4f}".format(x_min, x_max))
print("{:.4f} < Y < {:.4f}".format(y_min, y_max))

df_train['x_entry'], df_train['x_exit'] = normalise_X(df_train['x_entry']), normalise_X(df_train['x_exit'])
df_train['y_entry'], df_train['y_exit'] = normalise_Y(df_train['y_entry']), normalise_Y(df_train['y_exit'])
df_test['x_entry'], df_test['x_exit'] = normalise_X(df_test['x_entry']), normalise_X(df_test['x_exit'])
df_test['y_entry'], df_test['y_exit'] = normalise_Y(df_test['y_entry']), normalise_Y(df_test['y_exit'])

Borders:
-1.0000 < X < 1.0000
-0.3000 < Y < 0.3000


In [6]:
# normalising time information
def convert_time(time_sting):
    hms = time_sting.split(":")
    seconds = int(hms[0])*60*60 + int(hms[1])*60 + int(hms[2])
    seconds = (seconds-15*60*60)/(10*60*60)
    return seconds

df_train["t_entry"] = df_train["time_entry"].apply(lambda x: convert_time(x))
df_train["t_exit"] = df_train["time_exit"].apply(lambda x: convert_time(x))
df_test["t_entry"] = df_test["time_entry"].apply(lambda x: convert_time(x))
df_test["t_exit"] = df_test["time_exit"].apply(lambda x: convert_time(x))

In [7]:
# obtaining metadata from IDs
df_train['tid_0'] = [tid.split("_")[-1] for tid in df_train['trajectory_id']]
df_train['tid_1'] = [tid.split("_")[-2] for tid in df_train['trajectory_id']]
df_test['tid_0'] = [tid.split("_")[-1] for tid in df_test['trajectory_id']]
df_test['tid_1'] = [tid.split("_")[-2] for tid in df_test['trajectory_id']]
df_train['tid_0'], df_test['tid_0'] = df_train['tid_0'].astype(int), df_test['tid_0'].astype(int)
df_train['tid_1'], df_test['tid_1'] = df_train['tid_1'].astype(int), df_test['tid_1'].astype(int)

In [8]:
# extract relevant infromation and rearrange
columns = ['hash','tid_0',
           't_entry','t_exit',
           'x_entry','y_entry','x_exit','y_exit',
           'vmax','vmin','vmean',
           'time_entry','time_exit',
           'trajectory_id','tid_1']
df_train = df_train[columns]
df_test = df_test[columns]

In [9]:
# tid_1 is likely the day of the month, this information may be useful
print(max([int(x) for x in df_test['tid_0']]), max([int(x) for x in df_test['tid_1']]))

54 31


In [10]:
hash_most_freq = df_train['hash'].mode().tail(1).item()
df_train.loc[df_train['hash'] == hash_most_freq]

,hash,tid_0,t_entry,t_exit,x_entry,y_entry,x_exit,y_exit,vmax,vmin,vmean,time_entry,time_exit,trajectory_id,tid_1
808281,fe1ebe07b2f3adca2e60e10f2d95099f_9,0,-1.439111,-1.439111,0.196111,-0.000211,0.196111,-0.000211,-1.00,-1.00,-1.00,00:36:32,00:36:32,traj_fe1ebe07b2f3adca2e60e10f2d95099f_9_0,9
808282,fe1ebe07b2f3adca2e60e10f2d95099f_9,1,-1.404222,-1.404222,0.196445,0.004345,0.196445,0.004345,-1.00,-1.00,-1.00,00:57:28,00:57:28,traj_fe1ebe07b2f3adca2e60e10f2d95099f_9_1,9
808283,fe1ebe07b2f3adca2e60e10f2d95099f_9,2,-1.369028,-1.369028,0.200341,0.004003,0.200341,0.004003,-1.00,-1.00,-1.00,01:18:35,01:18:35,traj_fe1ebe07b2f3adca2e60e10f2d95099f_9_2,9
808284,fe1ebe07b2f3adca2e60e10f2d95099f_9,3,-1.334472,-1.334472,0.196111,-0.000325,0.196111,-0.000325,-1.00,-1.00,-1.00,01:39:19,01:39:19,traj_fe1ebe07b2f3adca2e60e10f2d95099f_9_3,9
808285,fe1ebe07b2f3adca2e60e10f2d95099f_9,4,-1.300222,-1.300222,0.197447,0.006965,0.197447,0.006965,-1.00,-1.00,-1.00,01:59:52,01:59:52,traj_fe1ebe07b2f3adca2e60e10f2d95099f_9_4,9
808286,fe1ebe07b2f3adca2e60e10f2d95099f_9,5,-1.204111,-1.204111,0.197113,0.009357,0.197113,0.009357,-1.00,-1.00,-1.00,02:57:32,02:57:32,traj_fe1ebe07b2f3adca2e60e10f2d95099f_9_5,9
808287,fe1ebe07b2f3adca2e60e10f2d95099f_9,6,-1.064333,-1.064333,0.197781,0.006737,0.197781,0.006737,-1.00,-1.00,-1.00,04:21:24,04:21:24,traj_fe1ebe07b2f3adca2e60e10f2d95099f_9_6,9
808288,fe1ebe07b2f3adca2e60e10f2d95099f_9,7,-1.031722,-1.031722,0.201566,0.006395,0.201566,0.006395,-1.00,-1.00,-1.00,04:40:58,04:40:58,traj_fe1ebe07b2f3adca2e60e10f2d95099f_9_7,9
808289,fe1ebe07b2f3adca2e60e10f2d95099f_9,8,-0.966444,-0.966444,0.196000,0.004915,0.196000,0.004915,-1.00,-1.00,-1.00,05:20:08,05:20:08,traj_fe1ebe07b2f3adca2e60e10f2d95099f_9_8,9
808290,fe1ebe07b2f3adca2e60e10f2d95099f_9,9,-0.864306,-0.864306,0.198226,0.005028,0.198226,0.005028,-1.00,-1.00,-1.00,06:21:25,06:21:25,traj_fe1ebe07b2f3adca2e60e10f2d95099f_9_9,9


# BASELINE SUBMISSION

In [11]:
df_test_1st_traj_only = df_test[df_test['x_exit'].isnull()]
df_submit = df_test_1st_traj_only[['trajectory_id']].copy()
df_submit = df_submit.rename(columns = {'trajectory_id':'id'})

# helper function to determine if point is inside
def is_inside(arr_x, arr_y):
    return ((arr_x > x_min) & 
            (arr_x < x_max) & 
            (arr_y > y_min) & 
            (arr_y < y_max)).astype(float)

df_submit['target'] = is_inside(df_test_1st_traj_only['x_entry'],
                                df_test_1st_traj_only['y_entry'])
df_submit.to_csv('submission.csv', index=False)
df_submit.tail()

,id,target
202899,traj_ffe98f6e0adf12f9c7b51c4e9607a87a_15_13,0.0
202913,traj_fff607ecd3f8d3dcb65791e8b4c22a5f_3_25,1.0
202914,traj_fff813b56230c2f026f783f5b9f9ca90_19_0,0.0
202929,traj_fff9400843a88c3bfe52e7ce8bf97316_19_17,0.0
202936,traj_fff9552047b095e8242b4913f3289a26_25_7,0.0


# DATASET PIVOTING

In [12]:
p_train = df_train.pivot('hash', 'tid_0')
p_train.tail()

t_entry           ... tid_1    
tid_0                                      0         1  ...    65  66
hash                                                    ...          
fffd57d840cb6c16553cc8efc907ef6d_29 -0.322222 -0.304667 ...   NaN NaN
fffe0c49b72b5d066949457ea383a77e_3  -0.700111       NaN ...   NaN NaN
fffe926c252d7dfc79dec28c3a7ba0bb_3  -0.945639 -0.905194 ...   NaN NaN
ffff74ab47d4fa20695231ce8a293c98_1  -0.474583 -0.294972 ...   NaN NaN
ffffc6359725f0e1feac9ef1872ab207_11 -1.474278       NaN ...   NaN NaN

[5 rows x 845 columns]

In [13]:
p_test = df_test.pivot('hash', 'tid_0')
p_test.tail()

t_entry           ... tid_1    
tid_0                                      0         1  ...    51  54
hash                                                    ...          
ffe98f6e0adf12f9c7b51c4e9607a87a_15 -1.365694 -1.269750 ...   NaN NaN
fff607ecd3f8d3dcb65791e8b4c22a5f_3        NaN -1.414667 ...   NaN NaN
fff813b56230c2f026f783f5b9f9ca90_19  0.013389       NaN ...   NaN NaN
fff9400843a88c3bfe52e7ce8bf97316_19 -1.098333       NaN ...   NaN NaN
fff9552047b095e8242b4913f3289a26_25 -0.702889       NaN ...   NaN NaN

[5 rows x 689 columns]

In [14]:
def obtain_matrix(row):
    df_hash = row.stack().iloc[::-1].reset_index()
    trajectory_id = df_hash.loc[0,"trajectory_id"]
    df_hash = df_hash[['t_entry','t_exit',
                       'x_entry','y_entry','x_exit','y_exit',
                       'vmax','vmin','vmean','tid_0','tid_1']]
    targets = df_hash.loc[0,"x_exit"], df_hash.loc[0,"y_exit"]

    df_hash.loc[0,"x_exit"] = np.nan
    df_hash.loc[0,"y_exit"] = np.nan
    embeds = np.transpose(df_hash.values)
    df_hash = df_hash.append(pd.DataFrame([[np.nan]*df_hash.shape[1]], 
                                            columns=list(df_hash),
                                            index=[99]*(21-df_hash.shape[0])))
    return {"targets" : targets, 
            "df_hash" : df_hash,
            "matrix" : df_hash.values,
            "trajectory_id" : trajectory_id}

print(np.shape(obtain_matrix(p_train.iloc[[323]])["matrix"]))
print(obtain_matrix(p_train.iloc[[323]])["targets"])
obtain_matrix(p_train.iloc[[323]])["df_hash"]
# note that x_exit and y_exit is removed from matrix

(21, 11)
(-1.6122742073539644, -0.29982424471039326)


,t_entry,t_exit,x_entry,y_entry,x_exit,y_exit,vmax,vmin,vmean,tid_0,tid_1
0,0.028083,0.028083,-1.612274,-0.299824,NaN,NaN,NaN,NaN,NaN,17.0,19.0
1,-0.152417,-0.152417,-1.612274,-0.299824,-1.612274,-0.299824,NaN,NaN,NaN,16.0,19.0
2,-0.298111,-0.298111,-1.612274,-0.299824,-1.612274,-0.299824,NaN,NaN,NaN,15.0,19.0
3,-0.430444,-0.430444,-1.612274,-0.299824,-1.612274,-0.299824,NaN,NaN,NaN,14.0,19.0
4,-0.501750,-0.501750,-1.612274,-0.299824,-1.612274,-0.299824,NaN,NaN,NaN,13.0,19.0
5,-0.807222,-0.807222,-1.612274,-0.299824,-1.612274,-0.299824,NaN,NaN,NaN,11.0,19.0
6,-0.891333,-0.891333,-1.856843,-0.489747,-1.856843,-0.489747,NaN,NaN,NaN,10.0,19.0
7,-0.908806,-0.908806,-1.619621,-0.572908,-1.619621,-0.572908,NaN,NaN,NaN,9.0,19.0
8,-0.944778,-0.944778,-1.596022,-0.546938,-1.596022,-0.546938,NaN,NaN,NaN,8.0,19.0
9,-0.979806,-0.979806,-1.605150,-0.574633,-1.605150,-0.574633,NaN,NaN,NaN,7.0,19.0


In [15]:
test_data = []
test_ids = []

for i in tqdm(range(p_test.shape[0])):
    output = obtain_matrix(p_test.iloc[[i]])
    test_data.append(output["matrix"])
    test_ids.append(output["trajectory_id"])
#     if i>100:
#         break

In [16]:
print(np.shape(test_data))
print(np.shape(test_ids))
np.save("test_data", test_data)
np.save("test_ids", test_ids)

(33515, 21, 11)
(33515,)


In [17]:
train_data = []
train_targets = []

for i in tqdm(range(p_train.shape[0])):
    output = obtain_matrix(p_train.iloc[[i]])
    train_data.append(output["matrix"])
    train_targets.append(output["targets"])
#     if i>100:
#         break

In [18]:
# evaluate if the targets are inside
train_targets = np.array(train_targets)
train_targets_inside = is_inside(train_targets[:,0], train_targets[:,1])

In [19]:
print(np.shape(train_data))
print(np.shape(train_targets))
print(np.shape(train_targets_inside))
np.save("train_data", train_data)
np.save("train_targets", train_targets)
np.save("train_targets_inside", train_targets_inside)

(134063, 21, 11)
(134063, 2)
(134063,)


# TRAIN-TEST SPLIT INDICES

In [20]:
# standardised 4-fold train-test split for clustering purposes
from sklearn.model_selection import StratifiedKFold, KFold
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
trn_index_list = []
val_index_list = []
for trn_index, val_index in skf.split(np.arange(len(train_data)),
                                      train_targets_inside.astype(int)):
    trn_index_list.append(trn_index)
    val_index_list.append(val_index)
    
np.save("trn_index_list",trn_index_list)
np.save("val_index_list",val_index_list)

In [21]:
!ls

__notebook__.ipynb  test_ids.npy	      trn_index_list.npy
__output__.json     train_data.npy	      val_index_list.npy
submission.csv	    train_targets.npy
test_data.npy	    train_targets_inside.npy


In [22]:
# to document: specifications for all if not clear enough
# might not care to do: make the pivot table to 3D array faster